# Installs PyGrain (OSS only)

In [1]:
# @test {"output": "ignore"}
!pip install grain

# Imports

In [2]:
import grain.python as grain
import pprint

# `MapDataset`

`MapDataset` defines a dataset that supports efficient random access. Think of it as an (infinite) `Sequence` that computes values lazily. It will either be the starting point of the input pipeline or in the middle of the pipeline following another `MapDataset`. Grain provides many basic transformations for users to get started.

In [3]:
dataset = (
    grain.MapDataset.range(10)
    .shuffle(seed=10)  # Shuffles globally.
    .map(lambda x: x+1)  # Maps each element.
    .batch(batch_size=2)  # Batches consecutive elements.
)

pprint.pprint(dataset[0])
pprint.pprint(list(dataset))

array([6, 7])
[array([6, 7]), array([ 2, 10]), array([3, 5]), array([1, 4]), array([8, 9])]


The requirement for `MapDataset`'s source is a `grain.RandomAccessDataSource` interface: i.e. `__getitem__` and `__len__`.

In [4]:
# Note: Inheriting `grain.RandomAccessDataSource` is optional but recommended.
class MySource(grain.RandomAccessDataSource):
  def __init__(self):
    self._data = [0, 1, 2, 3, 4, 5, 6, 7]

  def __getitem__(self, idx):
    return self._data[idx]

  def __len__(self):
    return len(self._data)

In [5]:
source = MySource()

dataset = (
    grain.MapDataset.source(source)
    .shuffle(seed=10)  # Shuffles globally.
    .map(lambda x: x+1)  # Maps each element.
    .batch(batch_size=2)  # Batches consecutive elements.
)

pprint.pprint(dataset[0])
pprint.pprint(list(dataset))

array([6, 7])
[array([6, 7]), array([2, 8]), array([3, 5]), array([1, 4])]


Access by index will never raise an `IndexError` and can treat indices that are equal or larger than the length as a different epoch (e.g. shuffle differently, use different random numbers).

In [6]:
# Prints the 3rd element of the second epoch.
pprint.pprint(dataset[len(dataset)+2])

array([7, 3])


Note that `dataset[idx] == dataset[len(dataset) + idx]` iff there's no random transfomations. Since `dataset` has global shuffle, different epochs are shuffled differently:

In [7]:
pprint.pprint(dataset[len(dataset)+2] == dataset[2])

array([False, False])


You can use `filter` to remove elements not needed but it will return `None` to indicate that there is no element at the given index.

Returning `None` for the majority of positions can negatively impact performance of the pipeline. For example, if your pipeline filters 90% of the data it might be better to store a filtered version of your dataset.

In [8]:
filtered_dataset = dataset.filter(lambda e: (e[0] + e[1]) % 2 == 0)

pprint.pprint(f"Length of this dataset: {len(filtered_dataset)}")
pprint.pprint([filtered_dataset[i] for i in range(len(filtered_dataset))])

'Length of this dataset: 4'
[None, array([2, 8]), array([3, 5]), None]


`MapDataset` also supports slicing using the same syntax as Python lists. This returns a `MapDataset` representing the sliced section. Slicing is the easiest way to "shard" data during distributed training.

In [9]:
shard_index = 0
shard_count = 2

sharded_dataset = dataset[shard_index::shard_count]
print(f"Sharded dataset length = {len(sharded_dataset)}")
pprint.pprint(sharded_dataset[0])
pprint.pprint(sharded_dataset[1])

Sharded dataset length = 2
array([6, 7])
array([3, 5])


For the actual running training with the dataset, we should convert `MapDataset` into `IterDataset` to leverage parallel prefetching to hide the latency of each element's IO using Python threads.

This brings us to the next section of the tutorial: `IterDataset`.

In [10]:
iter_dataset = sharded_dataset.to_iter_dataset(grain.ReadOptions(num_threads=16, prefetch_buffer_size=500))

for element in iter_dataset:
  pprint.pprint(element)

array([6, 7])
array([3, 5])


# IterDataset

Most data pipelines will start with one or more `MapDataset` (often derived from a `RandomAccessDataSource`) and switch to `IterDataset` late or not at all. `IterDataset` does not support efficient random access and only supports iterating over it. It's an `Iterable`.

Any `MapDataset` can be turned into a `IterDataset` by calling `to_iter_dataset`. When possible this should happen late in the pipeline since it will restrict the transformations that can come after it (e.g. global shuffle must come before). This conversion by default skips `None` elements.

`DatasetIterator` is a stateful iterator of `IterDataset`. The state of the iterator can be cheaply saved and restored. This is intended for checkpointing the input pipeline together with the trained model. The returned state will not contain data that flows through the pipeline.

Essentially, `DatasetIterator` only checkpoints index information for it to recover (assuming the underlying content of files will not change).

In [11]:
dataset_iter = iter(dataset)
pprint.pprint(isinstance(dataset_iter, grain.DatasetIterator))

True


In [12]:
pprint.pprint(next(dataset_iter))

checkpoint = dataset_iter.get_state()

pprint.pprint(next(dataset_iter))

# Recover the iterator to the state after the first produced element.
dataset_iter.set_state(checkpoint)

pprint.pprint(next(dataset_iter))  # This should generate the same element as above

array([6, 7])
array([2, 8])
array([2, 8])
